# Consultas propuestas por el enunciado

#### Imports y definición de funciones

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

import re

import warnings
# suprimir future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
spark = SparkSession.builder\
    .master("local[*]") \
    .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/05 20:15:15 WARN Utils: Your hostname, LENOVOID-ubuntu, resolves to a loopback address: 127.0.1.1; using 192.168.0.235 instead (on interface wlp2s0)
25/10/05 20:15:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/05 20:15:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/05 20:15:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. ¿Cuál es el Estado que más descuentos tiene en total? ¿y en promedio?

#### Hipótesis
- Los estados con más descuentos son aquellos que tienen más órdenes registradas con un `discount_amount` no nulo y mayor a ceros.
- Se tienen en cuenta los estados de la columna `billing_address` de la tabla `orders`.
- Se considera que el estado está definido por un conjunto de dos letras mayúsculas seguido de un espacio y cinco dígitos (código postal).
- No se consideran los estados que no están definidos por el anterior patrón, ni tampoco las filas con un valor nulo en `billing_address`.

#### Limpieza de datos para la tabla de órdenes

In [ ]:
status_dict = {}
with open("status.txt", "r") as f:
    valid_statuses = [line.strip() for line in f.readlines()]
    id = 0
    for status in valid_statuses:
        status_dict[status] = id
        id += 1

bc_status = sc.broadcast(status_dict)

In [3]:
def state_and_postal_code(address):
    if address is None:
        return "UNDEFINED", None
    pattern = r'([A-Z]{2})\s(\d{5})'
    match = re.search(pattern, address)
    if match:
        return match.group(1), int(match.group(2))
    return "UNDEFINED", None

def retain_orders_columns(row: Row):
    state, postal_code = state_and_postal_code(row.billing_address)
    status = "UNDEFINED" if row.status is None else row.status.strip().upper()
    payment = "UNDEFINED" if row.payment_method is None else row.payment_method.strip().upper()
    return (
        row.customer_id,
        row.discount_amount,
        status,
        state,
        postal_code,
        payment
    )
    
ordersIdx = {
    "customer_id": 0,
    "discount_amount": 1,
    "status": 2,
    "state": 3,
    "postal_code": 4,
    "payment_method": 5,
}

In [4]:
orders = sqlContext.read.csv(
    'data/orders.csv',
    header=True, inferSchema=True
)
ordersRDD = orders.rdd.map(retain_orders_columns).cache()

#### Resolución

In [5]:
discountAndTotalOrdersByState = ordersRDD \
    .filter(lambda x: x[ordersIdx["state"]] != "UNDEFINED") \
    .map(
        lambda row: (
            row[ordersIdx["state"]], 
            (1 if (row[ordersIdx["discount_amount"]] is not None and row[ordersIdx["discount_amount"]] > 0) else 0, 1)
        )) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .cache()
# (estado, (#ordenes con descuentos, #ordenes totales))

In [6]:
highestCount = discountAndTotalOrdersByState\
    .reduce(lambda a, b: a if a[1][0] > b[1][0] else b)

highestAvg = discountAndTotalOrdersByState\
    .reduce(lambda a, b: a if a[1][0]/a[1][1] > b[1][0]/b[1][1] else b)

25/10/05 20:15:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, customer_id, order_date, status, payment_method, shipping_address, billing_address, discount_amount, tax_amount, shipping_cost, total_amount, currency, created_at, updated_at, subtotal
 Schema: _c0, order_id, customer_id, order_date, status, payment_method, shipping_address, billing_address, discount_amount, tax_amount, shipping_cost, total_amount, currency, created_at, updated_at, subtotal
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/orders.csv


In [7]:
print(f"El Estado con más órdenes con descuentos es {highestCount[0]} con {highestCount[1][0]} órdenes con descuentos.")

print(f"\nEl Estado con el mayor promedio de descuentos es {highestAvg[0]} con un promedio de {highestAvg[1][0]/highestAvg[1][1]:.2f}.")

El Estado con más órdenes con descuentos es AE con 30858 órdenes con descuentos.

El Estado con el mayor promedio de descuentos es KY con un promedio de 0.22.


### 2. ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado 'Refunded'? 
###    ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

#### Hipótesis
- Se tienen en cuenta los códgios postales de la columna `billing_address` de la tabla `orders`.
- Se considera que el código postal está definido por cinco dígitos precedido de un espacio y un conjunto de dos letras mayúsculas (estado).
- Se buscan los códigos postales con más apariciones entre las órdenes con estado `Refunded`
- Las órdenes con estado `Refunded` son todas aquellas que tienen como valor de la columna `status` el string `REFUNDED`, sin diferenciar mayúsculas de minúsculas.
- Para la búsqueda de nombres, se consideran a todos los usuarios de la tabla `customers` que tienen como valor de la columna `postal_code` alguno de los códigos postales encontrados.

#### Limpieza de datos para la tabla de usuarios

In [8]:
def retain_customers_columns(row: Row):
    first_name = "UNDEFINED" if row.first_name is None else row.first_name.strip().upper()
    segment = "UNDEFINED" if row.customer_segment is None else row.customer_segment.strip().upper()
    is_active = False if row.is_active is None else row.is_active
    marketing_consent = False if row.marketing_consent is None else row.marketing_consent
    return (
        row.customer_id,
        first_name,
        row.postal_code,
        segment,
        is_active,
        marketing_consent,
    )
    
customersIdx = {
    "id": 0,
    "name": 1,
    "postal_code": 2,
    "segment": 3,
    "is_active": 4,
    "consent": 5,
}

In [9]:
customers = sqlContext.read.csv(
    'data/customers.csv',
    header=True, inferSchema=True
)
customersRDD = customers.rdd.map(retain_customers_columns).cache()

#### Resolución

In [10]:
zipCodesWithMostRefundedOrdersCount = ordersRDD \
    .filter(
        lambda row: (
            row[ordersIdx["postal_code"]] is not None
            and row[ordersIdx["status"]] == "REFUNDED"
        )
    ) \
    .map(lambda row: (row[ordersIdx["postal_code"]], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(5, key=lambda x: -x[1])

topRefundedZipCodes = [postal_code for postal_code, _ in zipCodesWithMostRefundedOrdersCount]

In [11]:
mostFrecuentNameInTopRefundedZipCodes = customersRDD \
    .filter(
        lambda row: (
            row[customersIdx["postal_code"]] in topRefundedZipCodes
            and row[customersIdx["name"]] != "UNDEFINED"
        )
    ) \
    .map(lambda row: (row[customersIdx["name"]], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .reduce(lambda a, b: a if a[1] > b[1] else b)

25/10/05 20:15:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, email, first_name, last_name, phone, date_of_birth, gender, country, city, postal_code, address, registration_date, last_login, is_active, customer_segment, marketing_consent
 Schema: _c0, customer_id, email, first_name, last_name, phone, date_of_birth, gender, country, city, postal_code, address, registration_date, last_login, is_active, customer_segment, marketing_consent
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/customers.csv


In [12]:
print("Los 5 códigos postales con más órdenes reembolsadas son:")
print("\tCodigo\tÓrdenes reembolsadas")
for postal_code, count in zipCodesWithMostRefundedOrdersCount:
    print(f"\t{postal_code}\t{count}")

print(f"\nEl nombre más frecuente entre los clientes de esos códigos postales es: {mostFrecuentNameInTopRefundedZipCodes[0]}")

Los 5 códigos postales con más órdenes reembolsadas son:
	Codigo	Órdenes reembolsadas
	31571	6
	14396	5
	9045	5
	38151	5
	91623	5

El nombre más frecuente entre los clientes de esos códigos postales es: MICHAEL


### 3. Para cada tipo de pago y segmento de cliente, <br>devolver la suma y el promedio expresado como porcentaje <br>de clientes activos y de consentimiento de marketing

#### Hipótesis
- Se consideran valores únicos por combinación (usuario, método de pago). De esta forma no contamos dos compras del mismo usuario con el mismo método de pago.

#### Resolución

In [13]:
clients_formated = customersRDD.map(lambda row: (row[customersIdx["id"]], (row[customersIdx["segment"]], row[customersIdx["is_active"]], row[customersIdx["consent"]])))
orders_formated = ordersRDD.map(lambda row: (row[ordersIdx["customer_id"]], row[ordersIdx["payment_method"]]))

clients_orders = clients_formated.join(orders_formated)\
    .map(
        lambda x: (
            (x[0], x[1][1]), # KEY = (customer_id, payment_method)
            (x[1][0][1], x[1][0][2], x[1][0][0])     # VALUE = (is_active, consent, segment)
        )
    )
# (customer_id, ((segment, is_active, consent), payment_method))

# Me quedo con filas únicas por combinación de método de pago, customer_id
# para no contar dos veces a un mismo cliente que hizo varias órdenes con el mismo método de pago
# para un mismo customer_id, el segment is_active y consent siempre son los mismos
clients_orders_unique = clients_orders.reduceByKey(lambda a, _: a)

result = clients_orders_unique.map(
        lambda x: (
            (x[0][1], x[1][2]), # KEY = (payment_method, segment)
            (1 if x[1][0] else 0, 1 if x[1][1] else 0, 1)     # VALUE = (is_active, consent, count)
        )
    ).reduceByKey(
        lambda a, b: (
            a[0] + b[0], 
            a[1] + b[1], 
            a[2] + b[2]
        )
    ).map(
        lambda x: Row(
            payment_method=x[0][0],
            customer_segment=x[0][1],
            active_count=x[1][0],
            consent_count=x[1][1],
            active_percentage=x[1][0]/x[1][2] * 100,
            consent_percentage=x[1][1]/x[1][2] * 100,
        )
    )


In [14]:
df_result = spark.createDataFrame(result)
df_result.show()

+----------------+----------------+------------+-------------+-----------------+------------------+
|  payment_method|customer_segment|active_count|consent_count|active_percentage|consent_percentage|
+----------------+----------------+------------+-------------+-----------------+------------------+
|   BANK TRANSFER|          BUDGET|       16041|        12451|89.78004141713774| 69.68713270274807|
|     CREDIT CARD|          BUDGET|       16039|        12448|89.77889728519452| 69.67814161768821|
|CASH ON DELIVERY|         PREMIUM|       16437|        12807|89.87860892388451| 70.02952755905511|
|          PAYPAL|         REGULAR|       49138|        38306|89.96338337605273| 70.13181984621018|
|     CREDIT CARD|       UNDEFINED|        8232|         6367| 89.9770466717674| 69.59230516996394|
|       UNDEFINED|         REGULAR|       48441|        37783|89.93873004084664| 70.15038989974006|
|  DIGITAL WALLET|         REGULAR|       49127|        38299| 89.9663040691499| 70.13698128410797|


### 4. Para los productos que contienen en su descripción la palabra "stuff", <br> calcular el peso total de su inventario agrupado por marca, <br> mostrar sólo la marca y el peso total de las 5 más pesadas

#### Hipótesis
- Los productos que tienen un valor nulo en el peso no son considerados (peso nulo = 0kg).

#### Limpieza de datos para la tabla de productos

In [15]:
def retain_products_columns(row: Row):
    brand = "UNDEFINED" if row.brand is None else row.brand.strip().upper()
    weight = 0.0 if row.weight_kg is None else row.weight_kg
    stock = 0 if row.stock_quantity is None else row.stock_quantity
    is_stuff = False if row.description is None else ("STUFF" in row.description.upper())
    return (
        brand,
        weight,
        stock,
        is_stuff,
    )
    
productsIdx = {
    "brand": 0,
    "weight": 1,
    "stock": 2,
    "is_stuff": 3,
}

In [16]:
products = sqlContext.read.csv(
    'data/products.csv',
    header=True, inferSchema=True
)
productsRDD = products.rdd.map(retain_products_columns).cache()

#### Resolución

In [17]:
stuff_products_weight_by_brand = productsRDD \
    .filter(lambda x: x[productsIdx["is_stuff"]]) \
    .map(lambda x: (x[productsIdx["brand"]], x[productsIdx["weight"]])) \
    .reduceByKey(lambda a, b: a + b)
    
stuff_products_weight_by_brand.takeOrdered(5, key=lambda x: -x[1])

25/10/05 20:15:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_name, category_id, brand, price, cost, stock_quantity, weight_kg, dimensions, description, is_active, created_at
 Schema: _c0, product_id, product_name, category_id, brand, price, cost, stock_quantity, weight_kg, dimensions, description, is_active, created_at
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/products.csv


[('UNDEFINED', 23589.93),
 ('3M', 4250.86),
 ('WAYFAIR', 4080.17),
 ('ADIDAS', 4057.34),
 ('NIKE', 3614.96)]

### 5. Calcular el porcentaje de productos cuyo stock es al menos 20% <br> más alto que el stock promedio de su marca

#### Resolución

In [18]:
avg_stock_by_brand = productsRDD \
    .map(lambda x: (x[productsIdx["brand"]], (x[productsIdx["stock"]], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: x[0]/x[1])

result = productsRDD.map(lambda x: (x[productsIdx["brand"]], x[productsIdx["stock"]])) \
    .join(avg_stock_by_brand) \
    .mapValues(lambda x: (
        1 if x[0] > x[1]*1.2 else 0,  # productos con stock > 20% del promedio
        1
    ))\
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .map(
        lambda x: Row(
            brand=x[0],
            high_stock_products_percentage=x[1][0]/x[1][1] * 100
        )
    )

In [19]:
df_result = spark.createDataFrame(result)
df_result.show()

+---------------+------------------------------+
|          brand|high_stock_products_percentage|
+---------------+------------------------------+
|         MARVEL|             41.70086639306885|
|        CARTIER|            41.579673035257045|
|       PEDIGREE|             41.59448818897638|
|   UNDER ARMOUR|              41.8866291648637|
|     SONY MUSIC|             42.24486495228854|
|ELECTRONIC ARTS|            41.748333986671895|
|   OFFICE DEPOT|            41.512014065247115|
|         GIBSON|            41.732935719019224|
|       GOODYEAR|             41.47498221484467|
| LOCAL ARTISANS|            41.123439667128984|
|      TRAVELPRO|             41.50008260366761|
|        L'ORÉAL|             41.32231404958678|
|     EPIC GAMES|             41.58187751967748|
|        CASTROL|             42.18566392479436|
|         PURINA|             41.81565355582042|
|           TUMI|             41.40520694259012|
|       CETAPHIL|            41.259842519685044|
|   MICKEY MOUSE|   

### 6. Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos

#### Hipótesis
- Los productos más vendidos son aquellos que tienen mayor cantidad vendida (`quantity`) entre todas las órdenes que aperece

#### Limpieza y preparación de columnas de items

In [20]:
def retain_items_columns(row: Row):
    quantity = 0 if row.quantity is None else row.quantity
    return (
        row.order_id,
        row.product_id,
        quantity,
    )
    
itemsIdx = {
    "order_id": 0,
    "product_id": 1,
    "quantity": 2,
}

In [21]:
items = sqlContext.read.csv(
    'data/order_items.csv',
    header=True, inferSchema=True
)
itemsRDD = items.rdd.map(retain_items_columns).cache()

#### Resolución

In [22]:
top_products_counts = itemsRDD \
    .map(lambda x: (x[itemsIdx["product_id"]], x[itemsIdx["quantity"]])) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda x: -x[1])
top_products_ids = [product_id for product_id, _ in top_products_counts] # son solo 10 elementos

not_top_products_orders = itemsRDD \
    .map(lambda x: (x[itemsIdx["order_id"]], True if x[itemsIdx["product_id"]] in top_products_ids else False)) \
    .reduceByKey(lambda a, b: a or b) \
    .filter(lambda x: not x[1]) \
    .count()


25/10/05 20:16:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_item_id, order_id, product_id, quantity, unit_price, line_total, discount_amount
 Schema: _c0, order_item_id, order_id, product_id, quantity, unit_price, line_total, discount_amount
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/order_items.csv


In [23]:
print(f"La cantidad de órdenes que no contienen ninguno de los 10 productos más vendidos es: {not_top_products_orders}")

La cantidad de órdenes que no contienen ninguno de los 10 productos más vendidos es: 99507
